In [3]:
# %run ../routeLinkedList.ipynb #Uncomment to run tests
%run routeLinkedList.ipynb 

In [ ]:
import requests
import math
from geopy.distance import geodesic

In [2]:
REQUESTSTATES = {
    "WAITING":0,
    "SCHEDULED":1,
    "ONGOING":2,
    "PREDICTED":3,
    "COMPLETED":4,
    "REJECTED":5
    }

VEHICLESTATES = {
    "IDLE":0,
    "MOVING":1,
    "MOVINGFULL":2
}

# OSRM instance details
OSRM_HOST = "localhost"  # ROSRM server IP or hostname
OSRM_PORT = "5000"       # Port of OSRM server is listening on

In [ ]:
def calculateNewPosition(start_point, distance, bearing):
    # Radius of the Earth in meters
    R = 6371e3  
    bearing = math.radians(bearing)  # Convert bearing to radians

    lat1 = math.radians(start_point[0])  # Current lat point converted to radians
    lon1 = math.radians(start_point[1])  # Current long point converted to radians

    lat2 = math.asin(math.sin(lat1) * math.cos(distance / R) + 
                     math.cos(lat1) * math.sin(distance / R) * math.cos(bearing))

    lon2 = lon1 + math.atan2(math.sin(bearing) * math.sin(distance / R) * math.cos(lat1),
                             math.cos(distance / R) - math.sin(lat1) * math.sin(lat2))

    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)

    return (lat2, lon2)

In [ ]:
class Cords:
    def __init__(self, latidude, longitude):
        self.latidude = latidude
        self.longitude = longitude
    
    def __str__(self):
        return "lat: "+ str(self.latidude) + ",long: " + str(self.longitude)
    
    def getLatitude(self):
        return self.latidude
    
    def getLongitude(self):
        return self.longitude

In [3]:
class RequestCords:
    def __init__(self, latidude, longitude, start):
        self.latidude = latidude
        self.longitude = longitude
        self.start = start

    def __str__(self):
        return "lat: "+ str(self.latidude) + ",long: " + str(self.longitude) + ", " + str(self.start)
    
    def getLatitude(self):
        return self.latidude
    
    def getLongitude(self):
        return self.longitude
    
    def getStart(self):
        if self.start:
            return 1
        else:
            return 0

In [4]:
class Request:
    def __init__(self, passengerAmount, origin, destination, time):
        self.request_id = id(self)
        self.passengerAmount = passengerAmount
        self.origin = origin #RequestCords
        self.destination = destination #RequestCords
        self.timeOfRequest = time

    def __str__(self):
        return "Request: id: {}, passengerAmount: {}, origin: {}, destination: {}, time: {}".format(self.request_id, self.passengerAmount, self.origin, self.destination, self.timeOfRequest)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return self.request_id == other.request_id
    
    #Change state of request
    def changeState(self, state):
        self.state = REQUESTSTATES[state]

    #Getters
    def getId(self):
        return self.request_id
    
    def getPassengerAmount(self):
        return self.passengerAmount
    
    def getOrigin(self):
        return self.origin
    
    def getDestination(self):
        return self.destination
    
    def getTime(self):
        return self.timeOfRequest
    
    def getState(self):
        return self.state

In [5]:
class Routes:
    def __init__(self):
        self.routeList = RouteLinkedList()
        self.routeTime = 0
        self.idCounter = 0 #Used to give each request pair a unique id that is small in memory size

    def __str__(self):
        return str(self.routeList) + ",\n routeTime: " + str(self.routeTime)

    def __repr__(self):
        return self.__str__()

    def handleAddRequest(self, request, indexOrigin, indexDestination):
        if indexDestination < indexOrigin:
            raise ValueError("Tried to add a request to the routeList, but the destination happened before the pickup")
        self.routeList.insertAtIndex(request.origin, self.idCounter, indexOrigin)
        self.routeList.insertAtIndex(request.destination, self.idCounter, indexDestination)
        self.idCounter += 1
        return self.idCounter - 1
        # updateTime = self.updateTime()

    def handleNextArrival(self):
        cordPoint = self.routeList.deleteAtStart()
        if cordPoint == None:
            print("Tried to handle an arrival of a request, but there were no requests in the routeList")
            return None
        else:
            return cordPoint
        
    def getRequestOSRMJSON(self, vehiclePosition, steps=False):
        start_coords = str(vehiclePosition.longitude) + "," + str(vehiclePosition.latidude)
        middle_coords = ""
        #Traverse the linked list to get the coordinates
        while(route != None):
            middle_coords += str(route.cords.longitude) + "," + str(route.cords.latidude) + ";"
            route = route.next
        middle_coords = middle_coords[:-1]

        url = f"http://{OSRM_HOST}:{OSRM_PORT}/route/v1/driving/{start_coords};{middle_coords}?overview=false&steps={steps}"
        response = requests.get(url)

        # Parsing the response
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            Exception("Failed to get a response from the OSRM server")
    
    def calculateTotalDistance(self, vehiclePosition):
        data = self.getRequestOSRMJSON(vehiclePosition)
        distance = data['routes'][0]['distance']  # Distance in meters
        distance = round(distance)
        distance = distance/1000 # Convert meters to kilometers
        return distance

    def getListOfCords(self):
        #Check if the routeList is empty
        route = self.routeList.head
        if(route == None):
            print("Tried to get a list of cords from the routeList, but the routeList was empty")
            return 0
        
        cordsList = []
        #Traverse the linked list to get the coordinates
        while(route != None):
            cordsList.append(route.cords)
            route = route.next
        return cordsList

    def getRouteTotalTime(self, vehiclePosition):
        data = self.getRequestOSRMJSON(vehiclePosition)
        duration = data['routes'][0]['duration']  # Duration in seconds
        return duration
            
    def getSize(self):
        return self.routeList.getSize()
    
    def getRouteHead(self):
        return self.routeList.head

In [2]:
class Vehicle:
    def __init__(self, capacity, origin):
        self.vehicle_id = id(self)
        self.capacity = capacity
        self.currentAmountOfPassengers = 0
        self.currentPosition = origin
        self.state = VEHICLESTATES["IDLE"]
        self.route = Routes()
        self.onGoingRequestList = {}
        self.completedRequestList = {}

    def __str__(self):
        return "Vehicle: id: {}, capacity: {}, currentPosition: {}, route: {}".format(self.vehicle_id, self.capacity, self.currentPosition, self.route.getSize())
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return self.vehicle_id == other.vehicle_id
    
    def addRequestToRoute(self, request, indexOrigin, indexDestination):
        #Check Passengers Limit
        if request.getPassengerAmount() + self.currentAmountOfPassengers > self.capacity:
            print("Request exceeds capacity of vehicle")
        else:
            #If vehicle has space add request to the route at the specified indexes
            requestID = self.route.handleAddRequest(request, indexOrigin, indexDestination)
            request.changeState("SCHEDULED")
            self.onGoingRequestList[requestID] = request
            self.currentAmountOfPassengers += request.getPassengerAmount()
    
    def arrivedAtNextStop(self):
        cordPoint = self.route.handleNextArrival()
        if not cordPoint.cords.start:
            #Meaning a request has been completed
            completedRequest = self.onGoingRequestList[cordPoint.requestId]
            completedRequest.changeState("COMPLETED")
            del self.onGoingRequestList[cordPoint.requestId]  #Delete from onGoingRequestList
            
            self.currentPosition = cordPoint.cords
            self.currentAmountOfPassengers -= completedRequest.getPassengerAmount()
        else:
            #Meaning a request has been picked up
            self.onGoingRequestList[cordPoint.requestId].changeState("ONGOING")
            self.currentAmountOfPassengers += self.onGoingRequestList[cordPoint.requestId].getPassengerAmount()
            self.currentPosition = cordPoint.cords

    def updateRoute(self,timeDifference):
        route = self.route.getRouteHead()
        if route == None:
            return 0 #Route is empty, nothing to update 
        
        data = self.route.getRequestOSRMJSON(self.currentPosition, steps=False) #Get the data to check how many waypoints were completed
        numOfLegs = len(data['routes'][0]['legs'])

        routeTime = 0
        i = 0
        while i < numOfLegs:
            routeTime += data['routes'][0]['legs'][i]['duration']
            if routeTime > timeDifference:
                routeTime -= data['routes'][0]['legs'][i]['duration']
                break
            i += 1
        
        #Handle Arrival at destinations i
        for j in range(i):
            self.arrivedAtNextStop()
        
        timeLeft = timeDifference - routeTime

        if timeLeft < 0:
            Exception("Time left is negative, something went wrong")
        if timeLeft == 0:
            return
        
        #Handle the time left
        check = False
        data = self.route.getRequestOSRMJSON(self.currentPosition, steps=True)
        currentCord = self.currentPosition
        for j in range(len(data['routes'][0]['legs'][i]['steps'])):
            timeLeft -= data['routes'][0]['legs'][i]['steps'][j]['duration'] #Find the step that the vehicle is on
            if timeLeft < 0:
                timeLeft += data['routes'][0]['legs'][i]['steps'][j]['duration']
                currentCord = Cords(data['routes'][0]['legs'][i]['steps'][j-1]['maneuver']['location'][1], data['routes'][0]['legs'][i]['steps'][j-1]['maneuver']['location'][0])
                check = True
                break
        if check == False:
            Exception("Failed to find the step that the vehicle is on")
        
        #Estimate the current position of the vehicle based on the time left, bearings
        bearing = data['routes'][0]['legs'][i]['steps'][j]['maneuver']['bearing_after']
        distance = timeLeft * data['routes'][0]['legs'][i]['steps'][j]['distance'] / data['routes'][0]['legs'][i]['steps'][j]['duration']
        lat, lon = calculateNewPosition(currentCord, distance, bearing)

        currentCord = Cords(lat, lon)
        self.currentPosition = currentCord

    def routeDistance(self):
        return self.route.calculateTotalDistance(self.currentPosition)
    
    def routeTime(self):
        return self.route.getRouteTotalTime(self.currentPosition)

    def printRoute(self):
        return self.route
    
    def getPosition(self):
        return self.currentPosition
    
    def getPassengerAmount(self):
        return self.currentAmountOfPassengers
    
    def getRoute(self):
        return self.route
    
    def getBusCapacity(self):
        return self.capacity